In [28]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [29]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

Preprocess data

In [30]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

87 documents
22 classes ['dist_charminar1', 'dist_charminar2', 'dist_charminar3', 'dist_charminar4', 'dist_charminar5', 'goodbye', 'greeting', 'hotels_hyd', 'kukatpally1', 'kukatpally2', 'lbnagar1', 'lbnagar2', 'lbnagar3', 'lbnagar4', 'malls', 'name', 'resorts', 'secunderabad1', 'serilingampally1', 'serilingampally2', 'serilingampally3', 'temples']
62 unique lemmatized words ['a', 'am', 'and', 'anyone', 'are', 'between', 'bye', 'call', 'can', 'charminar', 'cya', 'day', 'distance', 'famous', 'give', 'good', 'have', 'hello', 'hi', 'hotel', 'how', 'hyderabad', 'i', 'in', 'is', 'khairatabad', 'know', 'kukatpally', 'l.b', 'later', 'lb', 'leaving', 'let', 'list', 'mall', 'me', 'meet', 'more', 'nagar', 'name', 'nice', 'of', 'resort', 'restaurant', 'say', 'secunderabad', 'see', 'serilingampally', 'shopping', 'should', 'some', 'talk', 'tell', 'temple', 'the', 'there', 'to', 'up', 'what', 'whats', 'you', 'your']


Create training and testing data

In [44]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) 
for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


# Build the model

In [32]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
#model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200
627/627 [==============================] - 1s 1ms/step - loss: 0.0863 - acc: 0.9777
Epoch 2/200
627/627 [==============================] - 0s 547us/step - loss: 6.4989e-05 - acc: 1.0000
Epoch 3/200
627/627 [==============================] - 0s 556us/step - loss: 6.3416e-05 - acc: 1.0000
Epoch 4/200
627/627 [==============================] - 0s 579us/step - loss: 1.2213e-04 - acc: 1.0000
Epoch 5/200
627/627 [==============================] - 0s 553us/step - loss: 7.0333e-04 - acc: 1.0000
Epoch 6/200
627/627 [==============================] - 0s 596us/step - loss: 1.7026e-04 - acc: 1.0000
Epoch 7/200
627/627 [==============================] - 0s 611us/step - loss: 1.6214e-06 - acc: 1.0000
Epoch 8/200
627/627 [==============================] - 0s 564us/step - loss: 3.9177e-04 - acc: 1.0000
Epoch 9/200
627/627 [==============================] - 0s 599us/step - loss: 2.7559e-05 - acc: 1.0000
Epoch 10/200
627/627 [==============================] - 0s 563us/step - loss: 7.5465e-07

627/627 [==============================] - 0s 580us/step - loss: 1.0425e-06 - acc: 1.0000
Epoch 81/200
627/627 [==============================] - 0s 531us/step - loss: 8.9066e-07 - acc: 1.0000
Epoch 82/200
627/627 [==============================] - 0s 547us/step - loss: 3.8430e-05 - acc: 1.0000
Epoch 83/200
627/627 [==============================] - 0s 552us/step - loss: 2.9318e-07 - acc: 1.0000
Epoch 84/200
627/627 [==============================] - 0s 539us/step - loss: 5.7091e-07 - acc: 1.0000
Epoch 85/200
627/627 [==============================] - 0s 542us/step - loss: 2.9597e-05 - acc: 1.0000
Epoch 86/200
627/627 [==============================] - 0s 553us/step - loss: 8.1002e-07 - acc: 1.0000
Epoch 87/200
627/627 [==============================] - 0s 534us/step - loss: 1.4916e-05 - acc: 1.0000
Epoch 88/200
627/627 [==============================] - 0s 558us/step - loss: 1.0377e-06 - acc: 1.0000 0s - loss: 1.1404e-06 - acc: 1.
Epoch 89/200
627/627 [==============================] 

627/627 [==============================] - 0s 559us/step - loss: 3.5375e-07 - acc: 1.0000
Epoch 159/200
627/627 [==============================] - 0s 536us/step - loss: 1.6560e-07 - acc: 1.0000
Epoch 160/200
627/627 [==============================] - 0s 536us/step - loss: 7.9515e-07 - acc: 1.0000
Epoch 161/200
627/627 [==============================] - 0s 555us/step - loss: 2.2635e-07 - acc: 1.0000
Epoch 162/200
627/627 [==============================] - 0s 534us/step - loss: 1.6712e-05 - acc: 1.0000
Epoch 163/200
627/627 [==============================] - 0s 533us/step - loss: 3.2560e-07 - acc: 1.0000
Epoch 164/200
627/627 [==============================] - 0s 555us/step - loss: 1.5752e-07 - acc: 1.0000
Epoch 165/200
627/627 [==============================] - 0s 543us/step - loss: 3.2816e-07 - acc: 1.0000
Epoch 166/200
627/627 [==============================] - 0s 542us/step - loss: 3.8482e-04 - acc: 1.0000
Epoch 167/200
627/627 [==============================] - 0s 552us/step - loss:

# Predict the response (Graphical User Interface)

In [33]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
#model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [34]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


In [35]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [36]:
#Creating GUI with tkinter
import tkinter
from tkinter import *
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)
#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)
ChatLog.config(state=DISABLED)
#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
#Create Button to send message
SendButton = Button(base, font=("Verdana",10,'bold'), text="Send", width="12", height="2",
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )
#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="30", height="2", font="Arial")
#EntryBox.bind("<Return>", send)
#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)
base.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\dilee\anaconda3\envs\tensorflow\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-36-ae06a881fea9>", line 12, in send
    res = chatbot_response(msg)
  File "<ipython-input-35-67b547fb1230>", line 10, in chatbot_response
    ints = predict_class(text, model)
  File "<ipython-input-34-ee1c17423c90>", line 24, in predict_class
    res = model.predict(np.array([p]))[0]
  File "C:\Users\dilee\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training.py", line 1149, in predict
    x, _, _ = self._standardize_user_data(x)
  File "C:\Users\dilee\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training.py", line 751, in _standardize_user_data
    exception_prefix='input')
  File "C:\Users\dilee\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training_utils.py", line 138, in standardize_input_data
    str(data_shape))
ValueError: Error whe